In [1]:
import os 
import pickle
import numpy as np
from os.path import join  

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, make_scorer

import tensorflow as tf
from tensorflow import keras
from tensorflow.distribute import MirroredStrategy
from tensorflow.keras import backend as K

datasets = join(os.getcwd(), "datasets")
preprocessed_datasets = join(datasets, "preprocessed")

X_all, y_all, X_test = pickle.load(open(join(preprocessed_datasets, "cleaned_ohe.pickle"), 'rb'))
X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, shuffle=True, train_size=.75, random_state=42)

strategy = MirroredStrategy(["GPU:0", "GPU:1"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [2]:
class BCR(tf.keras.metrics.Metric):
    def __init__(self, name='BCR', dtype=None):
        super().__init__(name, dtype=dtype)
        self.TP = tf.keras.metrics.TruePositives()
        self.TN = tf.keras.metrics.TrueNegatives()
        self.FP = tf.keras.metrics.FalsePositives()
        self.FN = tf.keras.metrics.FalseNegatives()
        
        self.bcr = self.add_weight(name='bcr', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        
        self.TP.update_state(y_true, y_pred)
        self.TN.update_state(y_true, y_pred)
        self.FP.update_state(y_true, y_pred)
        self.FN.update_state(y_true, y_pred)
        
        tp = self.TP.result() 
        tn = self.TN.result()
        fp = self.FP.result()
        fn = self.FN.result()
        
        self.bcr.assign(.5 * ((tp / (tp + fn)) + (tn / (fp + tn))))
    
    def result(self):
        return self.bcr
        
    def reset_states(self):
        self.bcr.assign(0.0)


def recall(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return tp / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    return tn / (tn + fp + K.epsilon())

def bcr(y_true, y_pred):
    return .5 * (recall(y_true, y_pred) + specificity(y_true, y_pred))

def p1(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    m = m1(y_true, y_pred)
    return tp / (m + K.epsilon())

def p2(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    m = m2(y_true, y_pred)
    return tn / (m + K.epsilon())

def m1(y_true, y_pred):
    return K.sum(K.round(y_true))

def m2(y_true, y_pred):
    return K.sum(K.round(K.clip(1 - y_true, 0, 1)))

def P(bcr, bcr_hat, p1, p2, m1, m2):
    bcr_delta = np.abs(bcr - bcr_hat)
    sigma = .5 * np.sqrt(((1-p1)*p1/m1) + (p2*(1-p2)/m2))
    return bcr - bcr_delta * (1 - np.exp(- bcr_delta / sigma))

def P_model(model, X_train, y_train, X_val, y_val, with_info=False):
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    bcr_ = bcr(y_train, y_train_pred)
    bcr_hat = bcr(y_val, y_val_pred)
    
    p1_ = p1(y_val, y_val_pred)
    p2_ = p2(y_val, y_val_pred)
    m1_ = m1(y_val, y_val_pred)
    m2_ = m2(y_val, y_val_pred)
    
    p = P(bcr_, bcr_hat, p1_, p2_, m1_, m2_)
    
    if with_info:
        return p, {'bcr':bcr_, 'bcr_hat':bcr_hat, 'p1':p1_, 'p2':p2_, 'm1':m1_, 'm2':m2_}
    
    return p
    

class BCREarlyStopping(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

      Arguments:
          patience: Number of epochs to wait after min has been hit. After this
          number of no improvement, training stops.
      """

    def __init__(self, patience=0, restore_best_weights=False):
        super(BCREarlyStopping, self).__init__()
        self.patience = patience
        self.restore_best_weights = restore_best_weights
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = .0
        
    def on_epoch_end(self, epoch, logs=None):
        bcr = logs.get("bcr")
        val_bcr = logs.get("val_bcr")
        p1 = logs.get("val_p1")
        p2 = logs.get("val_p2")
        m1 = logs.get("val_m1")
        m2 = logs.get("val_m2")
        current = P(bcr, val_bcr, p1, p2, m1, m2)
        
        if np.less(self.best, current):
            print(f"New best p value : {current}")
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                if self.restore_best_weights:
                    print(f"Restoring model weights from the end of the best epoch. Best value : {self.best:.3f}")
                    self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


In [3]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L2(0.0), activation='sigmoid')
    ])
    
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(lr=1e-3),
        metrics=[bcr, p1, p2, m1, m2, "accuracy"]
        # metrics=[bcr, "accuracy"]
    )

bcr_early_stop = BCREarlyStopping(patience=30, restore_best_weights=True)

callbacks = [
    # tf.keras.callbacks.EarlyStopping(monitor='val_bcr', patience=20, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_bcr', factor=.5, patience=10, verbose=2),
    bcr_early_stop
]

model.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 1306757   
Total params: 1,306,757
Trainable params: 1,306,757
Non-trainable params: 0
_________________________________________________________________


In [4]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    batch_size=128,
                    epochs=200,
                    callbacks=callbacks,
                   )

Epoch 1/200
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm 

/home/romaingrx/anaconda3/envs/ingi2262/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in double_scalars


4/4 [==============================] - 0s 128ms/step - loss: 1964.9074 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 11.9000 - m2: 18.8500 - accuracy: 0.6122 - val_loss: 1580.6410 - val_bcr: 0.5000 - val_p1: 0.0000e+00 - val_p2: 1.0000 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.7179
Epoch 3/200
4/4 [==============================] - 0s 125ms/step - loss: 1513.9815 - bcr: 0.5212 - p1: 0.1423 - p2: 0.9000 - m1: 12.0833 - m2: 18.6667 - accuracy: 0.6103 - val_loss: 2394.3289 - val_bcr: 0.5000 - val_p1: 1.0000 - val_p2: 0.0000e+00 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.2821
New best p value : 0.4999999701976776
Epoch 4/200
4/4 [==============================] - 0s 126ms/step - loss: 1688.3709 - bcr: 0.5000 - p1: 1.0000 - p2: 0.0000e+00 - m1: 12.8167 - m2: 17.9333 - accuracy: 0.4164 - val_loss: 527.3217 - val_bcr: 0.4948 - val_p1: 0.0000e+00 - val_p2: 0.9896 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.7051
Epoch 5/200
4/4 [==============================] 

4/4 [==============================] - 0s 132ms/step - loss: 3.2450 - bcr: 0.9597 - p1: 0.9656 - p2: 0.9539 - m1: 11.5667 - m2: 19.1833 - accuracy: 0.9594 - val_loss: 91.1391 - val_bcr: 0.6531 - val_p1: 0.6979 - val_p2: 0.6083 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.6923
Epoch 29/200
4/4 [==============================] - 0s 132ms/step - loss: 5.6995 - bcr: 0.9644 - p1: 0.9804 - p2: 0.9484 - m1: 12.3167 - m2: 18.4333 - accuracy: 0.9573 - val_loss: 72.8991 - val_bcr: 0.7344 - val_p1: 0.6458 - val_p2: 0.8229 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.8077
Epoch 30/200
4/4 [==============================] - 0s 129ms/step - loss: 3.6191 - bcr: 0.9736 - p1: 0.9589 - p2: 0.9883 - m1: 12.6500 - m2: 18.1000 - accuracy: 0.9762 - val_loss: 73.9735 - val_bcr: 0.7344 - val_p1: 0.6458 - val_p2: 0.8229 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.8077
Epoch 31/200
4/4 [==============================] - 0s 126ms/step - loss: 1.3706 - bcr: 0.9883 - p1: 0.9802 - p2: 0.996

Epoch 55/200
4/4 [==============================] - 0s 130ms/step - loss: 0.0999 - bcr: 0.9980 - p1: 1.0000 - p2: 0.9960 - m1: 12.0000 - m2: 18.7500 - accuracy: 0.9972 - val_loss: 75.4400 - val_bcr: 0.7010 - val_p1: 0.6458 - val_p2: 0.7563 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.7692
Epoch 56/200
4/4 [==============================] - 0s 131ms/step - loss: 0.0726 - bcr: 0.9982 - p1: 1.0000 - p2: 0.9964 - m1: 12.3500 - m2: 18.4000 - accuracy: 0.9972 - val_loss: 74.4387 - val_bcr: 0.7281 - val_p1: 0.6458 - val_p2: 0.8104 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.7949
Epoch 57/200
4/4 [==============================] - 0s 132ms/step - loss: 0.1585 - bcr: 0.9865 - p1: 0.9847 - p2: 0.9883 - m1: 11.9333 - m2: 18.8167 - accuracy: 0.9865 - val_loss: 73.6336 - val_bcr: 0.7344 - val_p1: 0.6458 - val_p2: 0.8229 - val_m1: 5.5000 - val_m2: 14.0000 - val_accuracy: 0.8077
Restoring model weights from the end of the best epoch. Best value : 0.770
Epoch 00057: early stopping


Get the performance of the model
---

In [5]:
from utils import Report

report = Report(model, X_train, y_train, X_val, y_val).to_stdout()

------------------ Report for Sequential -------------------

P score : 0.803
BCR     : 0.968
BCR hat : 0.797


Save the csv output and model
---

In [6]:
from utils import to_contest_csv

y_test = model.predict(X_test).round().astype(np.uint8).flatten()

report.to_contest_csv(f"dnn_linear_{report.p.numpy():.3f}", y_test, save_model=True)